In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision.all import *
from fastai.callback.fp16 import *
import torch
import wandb
from fastai.callback.wandb import WandbCallback

from icevision.data import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from ceruleanml.data import class_list
from ceruleanml.learner_config import (
    memtile_size,
    rrctile_size,
    run_list,
    final_px,
    classes_to_keep,
    get_tfms,
    wd,
    record_collection_train,
    record_collection_val,
    record_collection_test,
    model_name,
    num_workers,
)
run_list

In [ ]:
config = {
    'memtile_size': memtile_size,
    'rrctile_size': rrctile_size,
    'run_list': run_list,
    'final_px': final_px,
    'classes_to_keep': classes_to_keep,
    'weight_decay': wd,
    'num_workers': num_workers,
    'train_record_count': len(record_collection_train),
    'val_record_count': len(record_collection_val),
    'test_record_count': len(record_collection_test)
}

wandb.init(project='cv3', entity="skytruth", config=config)

In [ ]:
bs_d ={512:8, 256:16, 224:16, 128:32, 64:64} # Batch Size for each image size
lr_d = {512:1e-3, 256:1e-3, 224:1e-3, 128:1e-3, 64:1e-3} # Learning Rate for each image size

In [ ]:
splitter = FuncSplitter(lambda o: "val" in str(o.filepath.parent.parent.stem))

def get_image(record):
    # return record.load().img
    return record.load().img.split()[0].convert('L')

def get_mask(record):
    return generate_flattened_mask_array(record.load())

def generate_flattened_mask_array(record):
    # Extract necessary information from the record
    string_labels = record.detection.labels
    if not string_labels:
        return np.zeros(record.common.img_size, dtype=np.uint8)
    class_map = record.detection.class_map
    labels = np.array([class_map.get_by_name(label) for label in string_labels], dtype=np.uint8)
    masks = record.detection.mask_array.data
    
    # Broadcast labels to match the shape of masks and compute the weighted masks
    weighted_masks = masks * labels[:, np.newaxis, np.newaxis]
    
    # Take the maximum along the first dimension
    flattened_mask = np.max(weighted_masks, axis=0)
    
    return flattened_mask

In [ ]:
r = Dataset(record_collection_train)[6]
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].imshow(get_image(r))
axs[1].imshow(get_mask(r))
plt.show()

In [ ]:
# train_tfms, val_tfms = get_tfms(reduced_resolution_tile_size=final_px) # XXX ETHAN distinguish which tfms are batch_tfms and which are item_tfms


In [ ]:
cbs = [
    WandbCallback(log_model=True),
    # ShortEpochCallback(pct=0.1, short_valid=True), 
    EarlyStoppingCallback(min_delta=.0001, patience=20),
    TerminateOnNaNCallback(), 
    GradientAccumulation(8), 
    GradientClip(), 
    SaveModelCallback(), 
    ShowGraphCallback(),
    MixedPrecision(), # I'm used to this being a .to_fp() on a learner, rather than a callback???
    ]

In [ ]:
seg_dblock = DataBlock(
        blocks=(ImageBlock(PILImageBW), MaskBlock(codes=class_list)), # ImageBlock is RGB by default, uses PIL
        getters=[get_image, get_mask],
        splitter=splitter,
        batch_tfms=[*aug_transforms()],
        item_tfms=[Resize(final_px//4)]
    )
dls = seg_dblock.dataloaders(source=[r for r in record_collection_train+record_collection_val], batch_size=bs_d[final_px], verbose=False).to(device)
dls.show_batch()

In [ ]:
model = resnet18()
body = create_body(model, n_in=1, pretrained=True)
unet = DynamicUnet(body, n_out=7, img_size=(final_px, final_px))

In [ ]:
loss_func = CrossEntropyLossFlat(axis=1)

In [ ]:
learner = Learner(dls=dls, model=unet, loss_func=loss_func, cbs=cbs, lr=lr_d[final_px], wd=wd, metrics=[DiceMulti(), foreground_acc])
learner.to(device)

from torchsummary import summary
# summary(learner.model, (1,512,512))

In [ ]:
learner.fit_one_cycle(10) # XXX TARGET FOR TONIGHT

In [ ]:
learner.show_results()

In [ ]:
wandb.finish()

In [ ]:
start_new = True
load_model_name = False

if start_new:
    print("Starting from scratch")
    learner.save("model")
elif load_model_name:
    print(f"Loading {load_model_name}")
    learner.load(load_model_name)
    learner.save("model")
else:
    print("Continuing current training session")
    learner.load("model")
    # export_scripted_model(learner)

In [ ]:
from datetime import datetime
from ceruleanml.inference import save_fastai_model_state_dict_and_tracing

def export_scripted_model(learner, model_name):
    dateTimeObj = datetime.now()
    timestampStr = dateTimeObj.strftime("%Y_%m_%d_%H_%M_%S")
    experiment_dir =  Path(f'/root/experiments/cv3/{timestampStr}_{model_name}_unet/')
    experiment_dir.mkdir(exist_ok=True)
    print(experiment_dir)
    save_template = "model.pt"
    save_fastai_model_state_dict_and_tracing(learner, save_template, experiment_dir) # XXX Ethan need to check this swap works

In [ ]:
def set_encoder_state(learner,trainable=True):
    state = "Unfreezing" if trainable else "Freezing"
    encoder_layers = learner.model[0]
    num_params = sum(p.numel() for p in encoder_layers.parameters())
    full_num_params = sum(p.numel() for p in learner.model.parameters())

    print(state, num_params, 'encoder parameters out of', full_num_params,'total parameters')
    for param in encoder_layers.parameters():
        param.requires_grad = False

In [ ]:
set_encoder_state(learner)

In [ ]:
running_total_epochs = {}

for size, epochs in run_list:
    # train_tfms, val_tfms = get_tfms(reduced_resolution_tile_size=size) # XXX ETHAN distinguish which tfms are batch and which are item
    # seg_dblock = DataBlock(
    #         blocks=(ImageBlock, MaskBlock(codes=class_list)), # ImageBlock is RGB by default, uses PIL
    #         n_inp=1,
    #         splitter=splitter,
    #         get_x=get_image_by_record_id,
    #         get_y=get_mask_by_record_id,
    #         batch_tfms=batch_tfms,
    #         item_tfms=item_tfms, # XXX ETHAN if you find a cheaper easier way to push new TFMS into an existing DLS, then we don't need to fully recreate these here... learner.dls.add_tfms()
    #     )

    # learner.dls = seg_dblock.dataloaders(source=train_val_record_ids, batch_size=bs_d[size])
    learner.dls.add_tfms(.......) # XXX ETHAN to explore?
    print("PR: Starting from running total", running_total_epochs)
    print("PR: image size", size)
    print("PR: epochs", epochs)

    learner.fine_tune(epochs, lr_d[size], freeze_epochs=0)

    running_total_epochs[size] = sum(filter(None,[running_total_epochs.get(size),epochs]))
    learner.save(model_name)
    export_scripted_model(learner, model_name)

torch.cuda.empty_cache()

In [ ]:
learner.show_results()

In [ ]:
inputs, targets = learner.dls.train.one_batch()

In [ ]:
targets.shape